In [2]:
%load_ext autoreload

In [3]:
import os
import sys
sys.path.append('../src-py/')
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [5]:
%autoreload
import prompted_conclusion_utils as conc_utils

In [6]:
valid_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_with_targets_with_arglexrank_conclusions.pkl')

#### Mining important concepts from posts (EXAMPLE):

In [6]:
post = ['while i ve been pro choice for as long as i have been aware of the existence of abortion, i can somewhat sympathize with elements of the pro life mindset.',
 'after all, if you truly believed that a fertilized egg is the same thing as a baby, then naturally you d regard abortion as child murder, and being angry hateful towards perceived murderers of children is pretty understandable.',
 'under that mindset, taking political action to ensure as few abortions as possible is sensible and relatable.',
 'where i lose that sympathy is in seeing the sum total of their political goals, related back to the idea that abortion is essentially child murder.',
 'for example, expanded access to contraceptives and comprehensive sex education are both shown to reduce the rates of both unintended pregnancy and abortion.',
 'if i truly believed that every abortion was a murdered baby, one could reasonably expect me to support pretty much any policy that would decrease the number of abortions, including these two.',
 'yet most of the same people who equate abortion with murder also support policies that restrict sex education and access to contraceptives, which go against the fundamental idea of preventing abortions.',
 'my belief is that pro lifers want fewer abortions only in the context of an overall socially conservative atmosphere, of which fewer abortions is just one small part.',
 'i believe that this worldview is more concerned with not being the sort of society where on demand abortion is legal than it is with the actual abortions themselves.']

In [7]:
post_nps = conc_utils.get_wiki_concepts([post])

TermWikifierClient: 100%|██████████| 71/71 [00:00<00:00, 139.72it/s]

In [9]:
post_nps

[['Birth control', 'Social conservatism', 'Anti-abortion movement']]

TermWikifierClient: 100%|██████████| 71/71 [00:20<00:00, 132.04it/s]

#### Generate multiple conclusions (EXAMPLE):

In [15]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":20,
    "top_p":0.95, 
    "num_beams":1
}

In [27]:
conc_utils.generate_prompted_conclusions([post],['It is'], gen_kwargs)

  0%|          | 0/1 [00:00<?, ?ba/s]

['hop musicipH with someones rythms is generic and boring']

In [16]:
post = ['i find most rap so incredibly mindnumbingly boring',
 "it's mostly repetitive beats with some poetry rapped over it and maybe a sung chorus in it for good measure",
 "problem is that for the most part it just lacks any real sense of melody and the rythms are incredibly basic 4 4 beats that never change throughout the song and as such there really isn't anything interesting or earcatching to latch onto musically",
 "amp; x200b; i've heard some people say that rap is supposed to be about the lyrics and not the music, but that just begs the question why it couldn't be published as some poetry record or spoken word album if the music simply is just filler to get some poetry out there",
 'of course there are some exceptions to this, some rap artists have more interesting beats that actually have something going on musically, the flow changes depending on the mood that section of the song is trying to convey and sometimes they just use a real band for the actual music but this feels like a rarity for the genre in general',
 "most of the lyrics also feel really basic and childish, even some of the more 'deeper' and well written lyrics that others recommend me just end up boiling down to the most baseline social critiques and an unhealthy obsession with penises in some tired and childish sexual metaphor",
 "amp; x200b; i've even heard people say that it's not about the music or the lyrics that's the point in some of these songs but that they try to get you pumped up and hype but i don't really find that kinda music very hype inducing, especially when it's just some simple beat that my dj friend could throw together in ten minutes that they just mumble or lazily yell over",
 "even the live shows are lame with the actual album track, vocals and all, being used as the backtrack while the rapper kinda half hardheartedly yells or mumbles and tries to get the crowd to be hyped, that's not hype, screaming into a guitar or bass mic on the fly to get a weird vocal effect is hype, screaming your lungs out to be heard over the band backtrack cause the mic broke is hype, having just the album track playing is not hype, it's lazy and it feels like people are just celebrating mediocrity"]

In [17]:
conc_utils.generate_prompted_conclusions([post],['Hip hop music'], gen_kwargs)

  0%|          | 0/1 [00:00<?, ?ba/s]

['Hip hop music is a really boring, uninteresting music, and should not be crit']

### Generate multiple conclusions for development dataset:

In [13]:
#sample_df = valid_df[valid_df.post.str.len() < 50].sample(10)

In [13]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":20,
    "top_p":0.95, 
    "num_beams":1
}

def generate_multiple_conclusions(df):
    posts_conclusions = []
    for i in range(6):
        control_codes = [x[i] if len(x) > i else '' for x in df['post_concepts'].tolist()]
        conclusions   = conc_utils.generate_prompted_conclusions(df['post'].tolist(), control_codes, gen_kwargs, batch_size=2)
        posts_conclusions.append(zip(control_codes, conclusions))

    df['post_conclusions'] = list(map(list, zip(*posts_conclusions))) # transpose
    
    return df

In [ ]:
df['post_concepts'] = conc_utils.get_wiki_concepts(df['post'].tolist(), max_concepts=5)
df = generate_multiple_conclusions(df)

In [53]:
df[['title', 'post', 'post_concepts', 'post_conclusions']].head(n=100)

,title,post,post_concepts,post_conclusions
365212,It is understandable that people put their religious rules above the law.,"[it is understandable that people put their religious rules above the law., when looking to christianity, people have only got this lifetime to live by their god's rules and after which their faith will be decided., a christian wants to live by god's rules and an islam wants to live by the rules created by allah because they believe their rules are the right one and that nobody can create rules better than them because he knows god allah what's best., if the christians and islams don't live by their god's rules they can be forgiven., but you can't really say: 'meh, i won't do this, i'll be forgiven anyway'., even when you are forced not to live by god's rules, as a believer in god, you know that god's rules are better and therefore you follow them.]",[],"[(, A religion should not give a voice to religion if their beliefs go against the law), (, you shouldn't have to live by a god's rules), (, it's understandable that people put their religious rules above the law.), (, I believe people should be free to choose the religion they choose), (, i generis and the other religion and then are for good reason.), (, You have to live by god's rules (or your faith).)]"
364114,the world will enter an era of practically eternal oppression,"[people who are motivated to help others tend to enter fields where they can help others such as medicine., people who are sociopathic and selfish tend to enter fields where they can gain power such as politics and business., unethical and evil tactics such as blackmail, lying, and killing give massive advantages towards rising to the top and gaining power., therefore, sociopathic and evil people tend to rise to positions of power as genuinely good people are not capable of competing., in the past, dictatorships have been overturned through revolutions, wars, and the death of leaders., most people are inherently followers so in order to start a revolution there must be a leader figure., it also takes planning to start a revolution., with modern day technology, the government can spy on people through cell phones and computers., facial recognition technology combined with gps on cell phones can ensure that they have information on the whereabouts of everyone at any time., for that reason, it is impossible for a leader figure to incite a revolution because the leader figure will be silenced before they can do anything., but what if a revolution does get going?, if it does, it has no hope of succeeding against an authoritatian government in the future., automated drones, tanks, and missiles already exist., the direction we are headed in is automated warfare., the government need only have control of an army of automated weapons to completely destroy any sort of civilian uprising., in the past, there was a possibility of the police and military also turning on the government., that possibility will not exist once the military is comprised mainly of machines., but what about the natural death of dictators?, given enough time someone better should come along and change things?, in the future, it is entirely possible that near immortality will be achievable through better medicine and genetic ing., leaders will stay in power for very long periods of time.]","[Mobile phone, Manner of death, Facial recognition system]","[(Mobile phone, Mobile phone in the classroom would drastically improve the ability of leadership figures to cause a revolution.), (Manner of death, Manner of death and medicine is the ultimate cause of humanity), (Facial recognition system, Facial recognition system's advancement in technology will lead to practically never a revolution.), (, i Ammoral people are the gatekeepers to our society), (, i think that people will rise to the top by killing, manipulating, and), (, people will continue to remain in power (long after they die).)]"
351525,"political correctness has gon

In [31]:
df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_multi_conclusions.pkl')

1499

### Run on Test:

In [17]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_sample_with_targets_with_arglexrank_conclusions.pkl')

In [ ]:
test_df['post_concepts'] = conc_utils.get_wiki_concepts(test_df['post'].tolist(), max_concepts=5)
test_df = generate_multiple_conclusions(test_df)


TermWikifierClient: 100%|██████████| 28/28 [00:00<00:00, 88.46it/s]

TermWikifierClient:   0%|          | 0/15 [00:00<?, ?it/s]

TermWikifierClient: 100%|██████████| 15/15 [00:01<00:00, 10.83it/s]

TermWikifierClient: 100%|██████████| 121/121 [00:00<00:00, 187.39it/s]

TermWikifierClient:   0%|          | 0/25 [00:00<?, ?it/s]

TermWikifierClient: 100%|██████████| 25/25 [00:00<00:00, 31.47it/s]


TermWikifierClient:   0%|          | 0/65 [00:00<?, ?it/s]


TermWikifierClient: 100%|██████████| 65/65 [00:00<00:00, 73.63it/s]



TermWikifierClient:   0%|          | 0/50 [00:00<?, ?it/s]



TermWikifierClient: 100%|██████████| 50/50 [00:00<00:00, 73.02it/s]




TermWikifierClient:   0%|          | 0/19 [00:00<?, ?it/s]




TermWikifierClient: 100%|██████████| 19/19 [00:00<00:00, 20.82it/s]





TermWikifierClient:   0%|          | 0/24 [00:00<?, ?it/s]





TermWikifierClient: 100%|██████████| 24/24 [00:00<00:00, 26.91it/s]






TermWikifierClient:   0%|          | 0/51 [00:00<?, ?it/s]


In [22]:
test_df[['title', 'post', 'post_concepts', 'post_conclusions']].head(n=5)

title  \
533361           all drugs should be legalized, manufactured, and available for public use.   
496733         Illegal immigrants broke the law, they should be treated as such not saints.   
487043   It is valid for a man to be more jealous of men then of women (in sexual context).   
511391                                     Monarchy have no reason to exist in modern world   
500768                                Homosexuality is a fetish and nothing truly romantic.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_multi_conclusions.pkl')

### Manually evaluate whether prompt-conclusions are valid conclusions:

In [23]:
sample_test = test_df.sample(50)

In [25]:
sample_test['conclusions'] = sample_test.post_conclusions.apply(lambda concs: [x[1] for x in concs if x[0] != ''])

In [28]:
sample_test[['post', 'title', 'conclusions']].head()

,post,title,conclusions
512794,"[i recently got in an argument with someone defending loli's and i was shocked., loli's are sexual drawings of very underage girls where guys usually above 18 obsess over them, adore them, and just do weird things to them., for me it's not that much different than child porn., these are drawings of young girls and degenerate neck beards jerk off to them and like those specific pics because they are so young., why can't they just jerk off to any other hentai?, because they enjoy how young these girls are., i also see the argument of how it's just fantasy and not real., yes they are fake thank god but these guys are still jerking off to them because of there age., these guys still have the fantasy of having sex with underage girls and use loli's as a way to please that desire., how come the furry community out right bans bestiality drawings because they find it disgusting and horrible but the hentai community still supports drawings of underage girls?, i also hear the argument 'oh we don't want to have sex with them we just adore them and find them so cute! ', that's still weird for me., 30 year olds adoring drawings of 9 year olds., honestly i don't think there is something right in the head if you adore those pics., for me loli's serve no good for anyone and appeal to weird neck beards., i would love to hear what people have to defend loli's and actually see if you guys change my views on them.]","Loli hentai is disgusting, creepy, and should be banned","[Child pornography is bad because loli is disgusting., Minor (law) Adults in the Hentai community do not deserve praise or adoration, Ban (law) Hentai.(def.i) loli!, Furry fandom shouldn't allow loli's to be adored.]"
540248,"[humans have been adapting to earth for hundreds of thousands of years., it's air, it's water, it's weather, it's day night cycle ect., even before 'modern' humans arrived our ancestors from which we evolved roamed this planet and did the same., so you could say we have spent over a billion years 'synchronizing' with this planet and it's soil., if you suddenly were to go to mars , it would completely throw everything out of whack., human's did not grow to live on that alien planet., the first generation of people living on mars will probably suffer great emotional, psychological and even physical pain attempting to live on this unfamiliar celestial body., stay here, where you have literally been born to live.]",You Shouldn't Want To Live On Mars Or Any Other Planet,[Astronomical objectification is wrong and human impulse to explore Mars is wrong.]
489096,"[in my opinion, the ownership of a company should always lie with the people employed by that company., the profit of the company should be given to those employees, and they should be able to elect the management., reason 1: it would democratise one of the most important aspects in most people s lives., people at hugely influenced by their workplace, yet those workplaces are almost always authoritarian in nature., we should instead expand the democracy to more parts of our lives, including the workplace., reason 2: it creates a fairer distribution of wealth., at the moment, you can buy shares of a company, and then get a part of the companies profit., but you have done no work to get those profits, you have added nothing to society., you are essentially getting money for already having money in the first place., on the other hand, you could give the profits to the people who actually worked for the money, who actually added value to society., reason 3: it would help with things like automation in the future., if the result of automation was not people losing their jobs, but instead people being able to work less while still having their job, that would be better., in our current economic system, automation will only benefit the few people who own parts in companies where jobs can be automated., if instead of losing jobs due to automation, we would lower the work times at 

In [29]:
sample_test[['post', 'title', 'conclusions']].to_csv('../data/prompted-conclusion-sample.csv')